In [8]:
from sklearn.tree import DecisionTreeRegressor  
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential  
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
arbol = DecisionTreeRegressor(max_depth=3)  # Evitar sobreajuste con árboles pequeños 

In [ ]:
def crear_red(input_shape):  
    model = Sequential([  
        Dense(16, activation='relu', input_shape=(input_shape,)),  
        Dense(1)  # Salida lineal para regresión  
    ])  
    model.compile(optimizer='adam', loss='mse')  
    return model  

# Envolver la red en un estimador Scikit-learn  
red_neuronal = KerasRegressor(  
    build_fn=lambda: crear_red(X_train.shape[1]),  
    epochs=50,  
    batch_size=32,  
    verbose=0  
)  

In [ ]:
class EnsembleSecuencial:  
    def __init__(self, estimadores, n_estimators=100, lr=0.1, sample_size=0.75):  
        self.estimadores = estimadores  # Lista de modelos base (ej: [arbol, red_neuronal])  
        self.n_estimators = n_estimators  
        self.lr = lr  
        self.sample_size = sample_size  
        self.modelos = []  

    def fit(self, X, y):  
        pred_actual = np.mean(y) * np.ones_like(y)  # Predicción inicial  
        for _ in range(self.n_estimators):  
            # 1. Calcular residuos  
            residuos = y - pred_actual  
            # 2. Muestreo aleatorio  
            idx = np.random.choice(len(X), int(len(X) * self.sample_size), replace=False)  
            X_muestra, residuo_muestra = X[idx], residuos[idx]  
            # 3. Entrenar modelo base (alternando entre árbol y red neuronal)  
            modelo = clone(self.estimadores[i % len(self.estimadores)])  # Alterna modelos  
            modelo.fit(X_muestra, residuo_muestra)  
            # 4. Actualizar predicciones  
            pred_actual += self.lr * modelo.predict(X)  
            # 5. Guardar modelo  
            self.modelos.append(modelo)  

    def predict(self, X):  
        pred = np.zeros(len(X))  
        for modelo in self.modelos:  
            pred += self.lr * modelo.predict(X)  
        return pred  

In [9]:
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

NameError: name 'X' is not defined

In [ ]:
# Inicializar el ensamble con árbol y red neuronal  
meta_modelo = EnsembleSecuencial(  
    estimadores=[DecisionTreeRegressor(), red_neuronal],  
    n_estimators=100,  
    lr=0.1  
)  

# Entrenar y evaluar  
meta_modelo.fit(X_train, y_train)  
predicciones = meta_modelo.predict(X_test)  

# Métricas  
from sklearn.metrics import r2_score  
print("R2:", r2_score(y_test, predicciones))  